In [ ]:
# 授权绑定Google Drive
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse
from google.colab import auth
auth.authenticate_user()
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

In [ ]:
# 指定Google Drive云端硬盘的根目录，名为drive
!mkdir -p drive
!google-drive-ocamlfuse drive

In [ ]:
import torch
import torchvision as tv
import torch.nn as nn
from tqdm import tqdm
import matplotlib.pyplot as plt


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("当前可用的设备为:{}".format(device))

In [ ]:
# 优化器选择
optimizers = {
    "sgd": torch.optim.SGD(net.parameters(), lr=0.01, momentum=0.9),
    "adagrad": torch.optim.Adagrad(net.parameters(), lr=0.1),
    "adadelta": torch.optim.Adadelta(net.parameters(), lr=0.01, rho=0.5, eps=1e-06, weight_decay=0),
    "adam": torch.optim.Adam(net.parameters(), lr=1e-2),
}

In [ ]:
data_transform = {
    "train": tv.transforms.Compose([tv.transforms.RandomCrop(28, padding=4),
                                    tv.transforms.RandomHorizontalFlip(),  # 随机翻转 (数据增强的操作)
                                    tv.transforms.ToTensor(),  # 转化为Tensor
                                    tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]),  # 标准化

    "test": tv.transforms.Compose([tv.transforms.ToTensor(),
                                   tv.transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
}

In [ ]:
train_dataset = tv.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transform['train'])
train_dataset, test_dataset, x = torch.utils.data.random_split(train_dataset, [10000, 5000, 35000])

train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=100, shuffle=True, num_workers=0)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=0)

In [ ]:
def model_run(net, train_loader, val_loader, optimizer, loss_function):
    best_accuracy = 0.0  # 最佳准确率
    train_steps = len(train_loader)
    train_Loss = []  # 训练集损失
    val_Loss = []  # 测试集损失
    Acc_Val = []  # 测试集的准确率
    Acc_Train = []  # 训练集的准确率
    val_num = 10000
    Epoch = 5  # 迭代次数
    num1 = 0
    num2 = 0
    for epoch in range(Epoch):
        # train
        net.train()
        running_loss = 0.0
        train_bar = tqdm(train_loader)  # 显示进度条函数
        score_train = 0
        running_loss = 0.0
        for step, data in enumerate(train_bar):
            images, labels = data
            optimizer.zero_grad()  # 清空梯度信息
            outputs = net(images.to(device))  # 将图片信息导入到GPU上
            pre_l = torch.max(outputs, dim=1)[1]
            score_train += (pre_l == labels.to(device)).sum().item()
            loss = loss_function(outputs, labels.to(device))  # 定义损失函数
            loss.backward()
            optimizer.step()
            running_loss += loss.item()  # print statistics
            num1 += 1

        train_bar.desc = "train epoch[{}/{}] |**| loss:{:.3f}".format(epoch + 1, Epoch, loss)
        train_Loss.append(running_loss/num1)
        Acc_Train.append(score_train / 5e4)

        # validation
        net.eval()
        accuracy_val = 0.0
        val_bar = tqdm(val_loader)  # 显示进度条函数
        val_running_loss = 0.0
        with torch.no_grad():  # 验证的时候不去更新梯度
            for val in val_bar:
                val_images, val_labels = val
                outputs = net(val_images.to(device))
                val_loss = loss_function(outputs, val_labels.to(device))
                predict_label = torch.max(outputs, dim=1)[1]
                accuracy_val += (predict_label == val_labels.to(device)).sum().item()
                val_running_loss += val_loss.item()
                num2 += 1

            acc_val = accuracy_val / val_num
            Acc_Val.append(acc_val)
            val_Loss.append(val_running_loss/num2)
            val_bar.desc = "Epoch[{}/{}] loss:{:.3f} test accuracy:{:.3f}%".format(epoch + 1, Epoch, val_running_loss, acc_val * 100)
            if acc_val > best_accuracy:
                best_accuracy = acc_val
        print('\n=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=-=\n')
        num1=0
        num2=0
    print("Finish Training!")
    return train_Loss, Acc_Train, val_Loss, Acc_Val

In [ ]:
def optim(device, optim):
    net = tv.models.resnet18(pretrained=False)
    net.fc = nn.Linear(512, 10)
    net.to(device)
    loss_function = nn.CrossEntropyLoss()
    train_Loss, Acc_Train, val_Loss, Acc_Val = model_run(net, train_loader, test_loader, optim, loss_function)
    return train_Loss

## SGD(Momentum)

In [ ]:
optim1 = optimizers["sgd"]
train_Loss1 = optim(device, optim1)

## AdaGrad

In [ ]:
optim2 = optimizers["adagrad"]
train_Loss2 = optim(device, optim2)

## AdaDelta

In [ ]:
optim3 = optimizers["adadelta"]
train_Loss3 = optim(device, optim3)

## Adam

In [ ]:
optim4 = optimizers["adam"]
train_Loss4 = optim(device, optim4)

## 损失函数下降曲线

In [ ]:
plt.figure(figsize=(5,8))
line1, = plt.plot(train_Loss1, color="deepskyblue", label="SGD(Momentum)")
line2, = plt.plot(train_Loss2, color='firebrick', label="Adagrad")
line3, = plt.plot(train_Loss3, color='deeppink', label="AdaDelta")
line4, = plt.plot(train_Loss4, color='darkorange', label="Adam")
first_legend = plt.legend(handles=[line1, line2, line3, line4], loc='upper right')
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.grid()
plt.title("Comparison between Two Optimizers")
plt.show()